In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import datetime

Clip Spatial

In [2]:
cycle = 14

kml_file = '/home/jovyan/shared-public/ICESat-2-Hackweek/ground_tracks/cycle%02i_1hz_points.pkl' % cycle
search_file = 'search_circle_ll.geojson'

geo_kml = pd.read_pickle(kml_file)
geo_search = gpd.GeoDataFrame.from_file(search_file)

# Clip gdf 'geo_kml' of kmls with gdf 'geo_search' of AOI (circle polygon)
geo_clipped = gpd.clip(geo_kml, geo_search)

# add linestrings to the file
linestrings = geo_clipped.groupby(['rgt'])['lat','lon'].apply(lambda x: LineString(list(zip(x.lon.tolist(),x.lat.tolist()))))
times = geo_clipped.groupby(by='rgt')['timestamp'].mean()
dflines = gpd.GeoDataFrame(times, geometry=linestrings, crs="EPSG:4326")
dflines['time_str'] = [datetime.datetime.strftime(datetime.datetime.fromtimestamp(t), '%a %Y-%b-%d %H:%M:%S') for t in dflines.timestamp]
dflines.reset_index(inplace=True)

# add the initial search area
geo = geo_search.copy()
geo['rgt'] = 'none'
geo['timestamp'] = 0
geo['time_str'] = 'none'

# put all of the dataframes together
df_out = pd.concat((geo, dflines, geo_clipped[dflines.keys()]))
df_out.reset_index(inplace=True, drop=True)
df_out.explore()

/tmp/ipykernel_838/3176873681.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  linestrings = geo_clipped.groupby(['rgt'])['lat','lon'].apply(lambda x: LineString(list(zip(x.lon.tolist(),x.lat.tolist()))))


In [3]:
# save it all to an output file
filename_out = search_file[:search_file.rfind('.')] + '_results.geojson'
df_out.to_file(filename_out, driver="GeoJSON") 
print('Wrote results to %s!' % filename_out)
df_out

Wrote results to search_circle_ll_results.geojson!


,geometry,rgt,timestamp,time_str
0,"POLYGON ((-48.85163 69.02742, -48.85528 69.005...",none,0.000000e+00,none
1,"LINESTRING (-49.37285 68.81711, -49.35174 68.8...",224,1.641423e+09,Wed 2022-Jan-05 22:53:26
2,"LINESTRING (-50.08191 69.02476, -50.06047 69.0...",285,1.641768e+09,Sun 2022-Jan-09 22:45:05
3,"LINESTRING (-49.50617 68.85196, -49.52734 68.9...",338,1.642067e+09,Thu 2022-Jan-13 09:51:29
4,"LINESTRING (-49.09508 68.87143, -49.07388 68.9...",666,1.643924e+09,Thu 2022-Feb-03 21:29:29
...,...,...,...,...
60,POINT (-49.24467 69.19627),224,1.641423e+09,Wed 2022-Jan-05 22:53:23
61,POINT (-49.50387 69.19752),1169,1.646769e+09,Tue 2022-Mar-08 19:57:09
62,POINT (-49.37830 69.24016),780,1.644568e+09,Fri 2022-Feb-11 08:27:35
63,POINT (-49.76063 69.20576),727,1.644269e+09,Mon 2022-Feb-07 21:21:06
